# CMIP6 ARD OHW2024

Date: 23 August, 2024

Author = {"name": "Thomas Moore", "affiliation": "CSIRO", "email": "thomas.moore@csiro.au", "orcid": "0000-0003-3930-1946"}

In [1]:
### setup dask cluster
from dask.distributed import Client
client = Client(threads_per_worker=1)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 2
Total threads: 2,Total memory: 9.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:35143,Workers: 2
Dashboard: /proxy/8787/status,Total threads: 2
Started: Just now,Total memory: 9.00 GiB
Comm: tcp://127.0.0.1:41867,Total threads: 1
Dashboard: /proxy/36497/status,Memory: 4.50 GiB
Nanny: tcp://127.0.0.1:33657,


In [2]:
import intake

In [3]:
cmip6_fs38_catalog = intake.open_esm_datastore("/g/data/dk92/catalog/v2/esm/cmip6-fs38/catalog.json")

In [4]:
cat_subset = cmip6.search(
source_id=['EC-Earth3'],
experiment_id=["ssp370"],
table_id="3hr",
variable_id="huss",
grid_label=["gn", 'gr'],
)

In [5]:
cat_subset

,unique
path,430
file_type,1
realm,1
frequency,1
table_id,1
project_id,1
institution_id,1
source_id,1
experiment_id,1
member_id,5


In [6]:
dset_dict = cat_subset.to_dataset_dict(
xarray_open_kwargs={"consolidated": True, "decode_times": True, "use_cftime": True}
)


--> The keys in the returned dictionary of datasets are constructed as follows:
	'file_type.project_id.institution_id.source_id.experiment_id.member_id.frequency.realm.table_id.variable_id.grid_label.version'


2024-07-29 11:09:25,018 - distributed.worker - WARNING - Compute Failed
Key:       _open_dataset-dfa03103-6da7-43da-9b98-196c2c62b8f6
Function:  execute_task
args:      ((<function apply at 0x149ab26d2c20>, <function _open_dataset at 0x149a970524d0>, ['/g/data/oi10/replicas/CMIP6/ScenarioMIP/EC-Earth-Consortium/EC-Earth3/ssp370/r13i1p1f1/3hr/huss/gr/v20200201/huss_3hr_EC-Earth3_ssp370_r13i1p1f1_gr_207501010000-207512312100.nc', 'huss'], (<class 'dict'>, [['xarray_open_kwargs', (<class 'dict'>, [['engine', 'netcdf4'], ['chunks', (<class 'dict'>, [])], ['backend_kwargs', (<class 'dict'>, [])], ['consolidated', True], ['decode_times', True], ['use_cftime', True]])], ['preprocess', None], ['expand_dims', (<class 'dict'>, [])], ['requested_variables', []], ['data_format', 'netcdf'], ['additional_attrs', (<class 'dict'>, [['path', '/g/data/oi10/replicas/CMIP6/ScenarioMIP/EC-Earth-Consortium/EC-Earth3/ssp370/r13i1p1f1/3hr/huss/gr/v20200201/huss_3hr_EC-Earth3_ssp370_r13i1p1f1_gr_207501010000-2

ESMDataSourceError: Failed to load dataset with key='f.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp370.r13i1p1f1.3hrPt.atmos.3hr.huss.gr.v20200201'
                 You can use `cat['f.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp370.r13i1p1f1.3hrPt.atmos.3hr.huss.gr.v20200201'].df` to inspect the assets/files for this key.
                 

In [ ]:
cat_subset.df['path'].values

# $The$ $End$

In [ ]:
client.shutdown()

## Plot current vectors for August

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)
import matplotlib.ticker as ticker
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy
from matplotlib import mlab, cm, gridspec
import matplotlib.ticker as mticker
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
%matplotlib inline 

In [ ]:
%%time
# Define the u and v components of the currents
time_choice = 8
u = clim_uv.u.sel(month=time_choice)
v = clim_uv.v.sel(month=time_choice)
speed = np.sqrt(u**2 + v**2)

In [ ]:
#plot model data
transform = ccrs.PlateCarree()
cmap = 'Spectral_r'
cbar_label='current speed'
plot_data = speed

###
fig = plt.figure(num=None, figsize=(8, 6), dpi=300, facecolor='w', edgecolor='k')
ax = plt.subplot(projection=ccrs.PlateCarree(180))
ax.set_extent([142,160, -25, -10], ccrs.PlateCarree())
ax.add_feature(cfeature.NaturalEarthFeature('physical', 'land', '50m', edgecolor='face', facecolor='white'))
ax.coastlines('50m',linewidth=0.5,edgecolor='grey')
plot_data.plot(transform=transform,cmap=cmap,cbar_kwargs={'label': cbar_label,'shrink':0.5},robust=True)

#plot u/v vectors
# Define the x and y coordinates
x = clim_uv.xu_ocean
y = clim_uv.yu_ocean
ax.quiver(x.values,y.values,u.values,v.values,transform=transform, units='x', width=0.01, scale=0.7, headwidth=2,alpha=0.2)
ax.set_title('BRAN2020 1993-2022\ncurrent speed \n August Climatology')